In [2]:
import ee
import geemap

In [3]:
Map = geemap.Map()

Enter verification code: 4/1AX4XfWjlKuN8rW6j9tD03UajXd3RgffhH-SwtLFZqT_pKDvnGnS_H1wXghM

Successfully saved authorization token.


In [4]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [7]:
#get the NLCD image 2016
NLCD = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
Map.addLayer(NLCD, {}, 'NLCD')

In [10]:
#look at the metadata
NLCD_meta = ee.FeatureCollection('users/giswqs/landcover/NLCD2016_metadata')
Map.addLayer(NLCD_meta, {}, 'NLCD Metadata')

In [11]:
#create a point for huntsville
point = ee.Geometry.Point([-86.58, 34.72])

In [13]:
#filter metadata to huntsville point
metadata = NLCD_meta.filterBounds(point).first()
region = metadata.geometry()

In [15]:
metadata.get('2016on_bas').getInfo()

'LC08_2016283'

In [17]:
doy = metadata.get('2016on_bas').getInfo().replace('LC08_', '')
doy

'2016283'

In [20]:
ee.Date.parse('YYYYDDD', doy).format('YYYY-MM-dd').getInfo()

'2016-10-09'

In [21]:
#filter landsat imagery by the point 
startdate = ee.Date.parse('YYYYDDD', doy)
enddate = startdate.advance(1, 'day')

In [44]:
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(point).filterDate(startdate, enddate).first().select('B[1-7]').clip(region)

In [27]:
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

In [28]:
Map.centerObject(point, 8)
Map.addLayer(image, vis_params, 'landsat-8')
Map

Map(bottom=3506.8182678222656, center=[34.720000000000006, -86.58], controls=(WidgetControl(options=['position…

In [29]:
#clip the NLCD to the study region. 
nlcd_raw = NLCD.clip(region)
Map.addLayer(nlcd_raw, {}, 'NLCD_clip')

In [30]:
#get the class values 
raw_class_values = nlcd_raw.get('landcover_class_values').getInfo()
print(raw_class_values)

[11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]


In [31]:
#remap the classes from 0-len of class values list. 
n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [34]:
class_palette = nlcd_raw.get('landcover_class_palette').getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [35]:
#create new, remaped landcover bands
nlcd = nlcd_raw.remap(raw_class_values, new_class_values).select(['remapped'], ['landcover'])
nlcd = nlcd.set('landcover_class_values', new_class_values)
nlcd = nlcd.set('landcover_class_palette', class_palette)

In [36]:
Map.addLayer(nlcd, {}, 'NLCD_remaped')
Map

Map(bottom=26179.0, center=[35.357696204467516, -84.94079589843751], controls=(WidgetControl(options=['positio…

In [40]:
#make the training data 
points = nlcd.sample(**{
    'region': region,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True
})

Map.addLayer(points, {}, 'training')

In [41]:
points.size().getInfo()

5000

In [42]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-85.76019437562952, 34.24361130908887]}, 'id': '0', 'properties': {'landcover': 8}}
